# Midterm Activity 1

> An assumption for this activity is that your team was tasked to provide recommendations based on your descriptive and predictive analysis of the assigned dataset.

### Objectives

1. Provide a descriptive analysis of a given dataset
2. Interpret the results of association rule mining metrics.
3. Apply association rule mining as the basis for providing recommendations.

In [6]:
# Import the dataset
import pandas as pd

themepark = pd.read_csv('themepark.csv')

themepark

,Timestamp,Do you agree with the statements above?,What age bracket do you belong to?,What is your gender?,What is the most common reason for you to visit a theme park?,How often do you visit a theme park?,"Do you have a specific preference for visiting theme parks during certain seasons, or are you open to visiting throughout the year?",At what time of the week do you typically visit a theme park?,During which quarter of the year do you most prefer to visit theme parks?,Do you plan your theme park visits around school holidays and breaks?,...,How often do you visit each food choice? [Fast Food],How often do you visit each food choice? [Restaurants],How often do you visit each food choice? [Food Trucks or Snacks],"Aside from Food, what other attractions do you visit in theme parks?",What kinds of merchandise do you usually purchase when visiting a theme park?,"How often do you visit each shop? [Souvenirs (fridge magnets, keychains...)]","How often do you visit each shop? [Apparel (hats, shirts...)]","How often do you visit each shop? [Gift Shop (plushies, toys...)]","How often do you visit each shop? [Photo Services (photo booths, meet-and-greet photos...)]","Aside from Shopping, what other attractions do you visit in theme parks?"
0,4/3/2024 21:30:10,I agree,26-40,Male,"Vacation, Family Outing",Once a year,I am open to visiting throughout the year,Both,NaN,NaN,...,1.0,1.0,4.0,"Shopping and Merchandise (Gift Shops, Souvenir...","Souvenirs (fridge magnets, keychains...), Appa...",3.0,3.0,3.0,1.0,None anymore
1,4/3/2024 21:43:03,I agree,18-25,Male,Family Outing,Once a year,I prefer visiting during specific seasons,Both,4th Quarter (Oct - Dec),Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/3/2024 21:43:20,I agree,18-25,Female,"Vacation, Special Occasion, Date",2-3 times a year,I am open to visiting throughout the year,Weekdays,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/3/2024 21:43:43,I agree,18-25,Male,Family Outing,Once a year,I prefer visiting during specific seasons,Weekdays,2nd Quarter (Apr - Jun),No,...,3.0,5.0,5.0,None anymore,NaN,NaN,NaN,NaN,NaN,NaN
4,4/3/2024 21:50:47,I agree,18-25,Male,"Vacation, Family Outing",Once a year,I prefer visiting during specific seasons,Both,4th Quarter (Oct - Dec),Yes,...,4.0,5.0,4.0,"Entertainment and Shows (Parades, Fireworks, C...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,4/7/2024 20:29:52,I agree,Under 18,Female,Vacation,Once a year,I am open to visiting throughout the year,Weekends,NaN,NaN,...,5.0,4.0,5.0,"Shopping and Merchandise (Gift Shops, Souvenir...","Gift Shop (plushies, toys...)",3.0,3.0,5.0,3.0,"Entertainment and Shows (Parades, Fireworks, C..."
109,4/7/2024 23:15:33,I agree,56 and over,Female,"Vacation, Family Outing, Special Occasion,",Once a year,I prefer visiting during specific seasons,Weekends,4th Quarter (Oct - Dec),Yes,...,NaN,4.0,4.0,"Entertainment and Shows (Parades, Fireworks, C...","Souvenirs (fridge magnets, keychains...), Appa...",3.0,4.0,3.0,3.0,"Food and Dining (Fast Food, Restaurants, Snacks)"
110,4/8/2024 6:29:52,I agree,18-25,Female,"Vacation, Family Outing",2-3 times a year,I am open to visiting throughout the year,Weekends,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,4/8/2024 8:22:34,I agree,18-25,Female,"Vacation, Special Occasion",Once a year,I am open to visiting throughout the year,Weekends,NaN,NaN,...,NaN,NaN,NaN,NaN,"Souvenirs (fridge magnets, keychains...), Phot...",4.0,2.0,3.0,5.0,None anymore


### Clean Data

Fill in missing data for 'What type of...' columns

In [7]:
rides = themepark[[
    'What kinds of rides do you visit?',
   'How often do you visit each type of ride? [Thrill Rides (High-speed roller coasters, Drop towers...)]',
   'How often do you visit each type of ride? [Family Rides (Carousels, Ferris wheels...)]',
   'How often do you visit each type of ride? [Kiddie Rides (Mini Coasters, Teacup rides...)]',
   'How often do you visit each type of ride? [Water Rides (Water slides, Splash pads...)]',
   'How often do you visit each type of ride? [Interactive Rides (Shooting galleries, Simulators...)]',
   'How often do you visit each type of ride? [Show Rides (Storytelling Rides, Horror Houses...)]'
]]

mask = rides.iloc[:, 1:].isna().all(axis=1)
rides.loc[mask, rides.columns[0]] = "I do not visit rides"

shows = themepark[[
    'What kinds of shows do you visit?',
   'How often do you visit each shows? [Stage shows (Live performances, Theatrical Productions...)]',
   'How often do you visit each shows? [Character Meet-and-Greets (Interact with characters from movies, cartoons and fairy tales)]',
   'How often do you visit each shows? [Parades (Floats, Dancers...)]',
   'How often do you visit each shows? [Nighttime Performances (Fireworks Display...)]'
]]

mask = shows.iloc[:, 1:].isna().all(axis=1)
shows.loc[mask, shows.columns[0]] = "I do not visit shows"

foods = themepark[[
    'What kinds of Food do you prefer when visiting a theme park?',
   'How often do you visit each food choice? [Fast Food]',
   'How often do you visit each food choice? [Restaurants]',
   'How often do you visit each food choice? [Food Trucks or Snacks]'
]]

mask = foods.iloc[:, 1:].isna().all(axis=1)
foods.loc[mask, foods.columns[0]] = "I do not buy food"

merchandise = themepark[[
    'What kinds of merchandise do you usually purchase when visiting a theme park?',
   'How often do you visit each shop? [Souvenirs (fridge magnets, keychains...)]',
   'How often do you visit each shop? [Apparel (hats, shirts...)]',
   'How often do you visit each shop? [Gift Shop (plushies, toys...)]',
   'How often do you visit each shop? [Photo Services (photo booths, meet-and-greet photos...)]'
]]

mask = merchandise.iloc[:, 1:].isna().all(axis=1)
merchandise.loc[mask, merchandise.columns[0]] = "I do not purchase merchandise"

### Combine all the types of attractions the visitors go to

In [11]:
columns_order = [
    'What attractions do you visit in theme parks?',
    'Aside from Entertainment, what other attractions do you visit in theme parks?',
    'Aside from Food, what other attractions do you visit in theme parks?',
    'Aside from Shopping, what other attractions do you visit in theme parks?'
]

def combine_attractions(row):
    included_values = []
    previous_values = []

    for col in columns_order:
        current_val = row[col]

        is_valid = (pd.notna(current_val) and (str(current_val).strip() != "") and (str(current_val) != "None anymore"))

        is_duplicate = current_val in previous_values

        if is_valid and not is_duplicate:
            included_values.append(str(current_val))

        previous_values.append(current_val)

    return ", ".join(included_values)

combined_attractions = themepark.apply(combine_attractions, axis=1)

combined_attractions

0      Rides (Roller coasters, Ferris wheels...), Foo...
1      Entertainment and Shows (Parades, Fireworks, C...
2              Rides (Roller coasters, Ferris wheels...)
3              Rides (Roller coasters, Ferris wheels...)
4      Food and Dining (Fast Food, Restaurants, Snack...
                             ...                        
108    Rides (Roller coasters, Ferris wheels...), Sho...
109    Shopping and Merchandise (Gift Shops, Souvenir...
110            Rides (Roller coasters, Ferris wheels...)
111            Rides (Roller coasters, Ferris wheels...)
112    Entertainment and Shows (Parades, Fireworks, C...
Length: 113, dtype: object

### Import ARM Dependencies and Prepare Data

In [30]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Split the products
attractions_list = list(combined_attractions.str.split(r',\s*(?![^()]*\))')  # Regex magic to skip commas in (...)
    .apply(
        lambda x: [item.strip() for item in x
                   if item.strip() not in ['', 'None Anymore']]
    ))

# One-hot Encoding
attractions_apr = TransactionEncoder()
apr_data = attractions_apr.fit(attractions_list).transform(attractions_list)
onehot_apr = pd.DataFrame(apr_data, columns=attractions_apr.columns_)

support_apr = apriori(onehot_apr, min_support = 0.2, use_colnames = True, verbose = 1)
support_apr

# Association Rules
attractions_ar = association_rules(support_apr, metric = "confidence", min_threshold = 0.6)

# Convert frozensets to sorted lists/strings
attractions_ar['antecedents'] = attractions_ar['antecedents'].apply(lambda x: ', '.join(sorted(list(x))))
attractions_ar['consequents'] = attractions_ar['consequents'].apply(lambda x: ', '.join(sorted(list(x))))

# Reorder columns for clarity
attractions_ar = attractions_ar[[
    'antecedents', 'consequents',
    'antecedent support', 'consequent support',
    'support', 'confidence', 'lift'
]]

attractions_ar

Processing 6 combinations | Sampling itemset size 32


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,"Entertainment and Shows (Parades, Fireworks, C...","Rides (Roller coasters, Ferris wheels...)",0.407080,0.867257,0.283186,0.695652,0.802130
1,"Food and Dining (Fast Food, Restaurants, Snacks)","Rides (Roller coasters, Ferris wheels...)",0.318584,0.867257,0.247788,0.777778,0.896825
2,"Shopping and Merchandise (Gift Shops, Souvenir...","Rides (Roller coasters, Ferris wheels...)",0.477876,0.867257,0.407080,0.851852,0.982237
